# Import

In [1]:
# !pip install imbalanced-learn
import imblearn
print(imblearn.__version__)

0.10.1


In [2]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import seaborn as sns
import statistics
import math as m

# import necessary packages
import pandas as pd
from copy import deepcopy as copy
import numpy as np
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
from torch import save as torch_save
from sklearn import datasets
from sklearn import preprocessing

from imblearn.over_sampling import SMOTE
from collections import Counter
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.metrics import confusion_matrix

# MODEL CREATION
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from torch.optim.lr_scheduler import CosineAnnealingLR, StepLR

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# BPNN Training and Testing
> ***Don't run this anymore unless you changed your dataset :D u already saved the model with the best performance as best_model2.pt on gdrive***

## EDA

In [4]:
data_dir = "/content/drive/MyDrive/MS thesis"
x = "/content/drive/MyDrive/MS thesis/preliminary site selection/CALABARZON/calabarzon_clusters.csv"
rg1n_clustered_df = pd.read_csv(x)
rg1n_clustered_df

,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Nearest_RHU,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster
0,0,2941561,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0.0,0,...,2971948,41.388247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
1,1,2942942,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0.0,0,...,2971948,41.271550,0.001364,0.000792,0.012129,0.006019,0.003191,0.002227,0.435149,0
2,2,2942943,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0.0,0,...,2971948,41.391750,4.724720,2.744833,42.027466,20.856238,11.058086,7.717034,0.392740,0
3,3,2949831,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1.0,0,...,2971948,22.516565,17.251052,6.571838,151.930272,75.921898,40.929798,27.108796,0.522489,0
4,4,2951212,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1.0,0,...,2971948,21.748555,38.483116,14.660254,338.921376,169.364234,91.304934,60.473468,0.569120,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,7503,3180648,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,1.0,0,...,3114331,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7504,7504,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0.0,0,...,3114331,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7505,7505,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0.0,0,...,3114331,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7506,7506,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0.0,0,...,3114331,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0


## Balance data
- this is so the model is properly trained with a more distributed set of data with 0s and 1s.


1. Downsampling data: to remove unbuildable areas
2. Dataframe set up: normalizing data
3. Upsampling data: generate data points with '1' label (just to teach the model)
4. Get low accessible areas (to turn into Candidate Sites)


### Downsampling

In [5]:
print(rg1n_clustered_df.shape)
print(rg1n_clustered_df['RHU_Presence'].unique())
print(rg1n_clustered_df['Cluster'].unique())
buildable = rg1n_clustered_df[rg1n_clustered_df['RHU_Presence'] == 1]
buildable

(7508, 22)
[0 1]
[0 1]


,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Nearest_RHU,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster
382,382,2970565,Gomez-Zamora (Pob.),Naic,Cavite,0.526595,1.477501,1.105369,0.0,1,...,2971948,0.0,658.745788,812.843865,8049.010116,4094.087924,2220.324693,1499.574238,0.501549,0
659,659,2981629,Calumpang Cerca,Indang,Cavite,0.249117,1.712915,1.203267,0.0,1,...,2983012,0.0,469.509009,517.216406,5661.398245,2885.228780,1531.448103,1089.525536,0.423709,0
783,783,2985759,Tabon III,Kawit,Cavite,0.771205,1.124705,1.046612,0.0,1,...,2987141,0.0,2962.826285,1935.937180,27920.958940,14055.187699,7679.249704,5337.458080,0.504555,0
804,804,2985780,Miguel Mojica,Mendez (Mendez-Nunez),Cavite,0.163139,1.702739,1.129446,0.0,1,...,2985780,0.0,1188.454618,1366.199792,12861.468384,6549.532980,3349.889788,2231.523366,0.441989,0
892,892,2988523,Toclong I-C,City of Imus,Cavite,0.807639,1.181400,0.962089,0.0,1,...,2987141,0.0,5208.886668,3445.871289,54059.595910,27916.684100,15925.633852,10166.067474,0.481689,0
893,893,2988524,Toclong I-C,City of Imus,Cavite,0.807639,1.181400,0.962089,0.0,1,...,2988524,0.0,2744.366649,2676.953573,30808.551635,15965.839701,8846.067473,5584.590459,0.469969,0
1060,1060,2992684,Tubuan II,Silang,Cavite,0.192312,1.914734,1.248136,0.0,1,...,2992684,0.0,655.532789,602.183440,7111.937777,3606.419727,1976.751228,1261.378692,0.417364,0
1194,1194,2996859,Sambat,San Pascual,Batangas,0.391973,1.954824,1.089212,0.0,1,...,2995477,0.0,1971.661763,1643.151842,20980.479651,10676.538575,5796.294412,3951.441305,0.447765,0
1350,1350,3002388,Kumintang Ilaya,Batangas City,Batangas,0.367578,1.855737,1.249972,0.0,1,...,3002389,0.0,1924.275195,1407.705376,20908.651113,10793.373861,6131.312720,4546.388991,0.487688,0
1481,1481,3006480,San Juan,Cainta,Rizal,0.836964,0.992960,1.004190,0.0,1,...,3005099,0.0,5008.969121,2168.984701,46581.737839,23458.744010,13373.494940,8901.105447,0.478396,0


In [6]:
# Drop rows with NaN values
rg1n_clustered_df.dropna(inplace=True) # DROP NANS so it doesnt interfere with the pcf calculations
rg1n_clustered_df['buildability_landcov'] = rg1n_clustered_df['buildability_landcov'].astype(int) # CONVERT LAND COV COL INTO INT FROM FLOAT
high_clustered = rg1n_clustered_df[rg1n_clustered_df['Cluster'] == 1]
high_clustered['buildability_landcov'].unique()
rg1n_clustered_df[rg1n_clustered_df['Cluster'] == 0]

''' If ever non buildable values come out for clusters with 1, change their values to buildable '''
# rg1n_clustered_df.loc[1079, ['buildability_landcov']] = [1]
# rg1n_clustered_df.loc[4676, ['buildability_landcov']] = [1]
# Filter rows with clusters of size 1
df_filtered = rg1n_clustered_df[rg1n_clustered_df['Cluster'] == 1]

if not df_filtered.empty:
    # Use loc[] for index-based updates
    df_filtered.loc[:, 'buildability_landcov'] = 1  # Update entire column with 1
    rg1n_clustered_df.update(df_filtered)
    print("Updated 'buildability_landcov' values to 1 for clusters with size 1.")
else:
    print("No clusters with size 1 found in the DataFrame.")
# print(rg1n_clustered_df)
rg1n_clustered_df

Updated 'buildability_landcov' values to 1 for clusters with size 1.


,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Nearest_RHU,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster
0,0,2941561,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.388247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
1,1,2942942,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.271550,0.001364,0.000792,0.012129,0.006019,0.003191,0.002227,0.435149,0
2,2,2942943,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.391750,4.724720,2.744833,42.027466,20.856238,11.058086,7.717034,0.392740,0
3,3,2949831,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,2971948,22.516565,17.251052,6.571838,151.930272,75.921898,40.929798,27.108796,0.522489,0
4,4,2951212,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,2971948,21.748555,38.483116,14.660254,338.921376,169.364234,91.304934,60.473468,0.569120,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,7503,3180648,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,1,0,...,3114331,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7504,7504,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7505,7505,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7506,7506,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0


In [7]:
'''DROP NON-BUILDABLE AREAS
>> dont do this anymore
'''
non_buildable = rg1n_clustered_df[rg1n_clustered_df['buildability_landcov'] == 0]
# rg1n_clustered_df.drop(rg1n_clustered_df.index[rg1n_clustered_df['buildability_landcov'] == 0], inplace = True)
rg1n_clustered_df

,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Nearest_RHU,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster
0,0,2941561,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.388247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
1,1,2942942,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.271550,0.001364,0.000792,0.012129,0.006019,0.003191,0.002227,0.435149,0
2,2,2942943,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.391750,4.724720,2.744833,42.027466,20.856238,11.058086,7.717034,0.392740,0
3,3,2949831,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,2971948,22.516565,17.251052,6.571838,151.930272,75.921898,40.929798,27.108796,0.522489,0
4,4,2951212,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,2971948,21.748555,38.483116,14.660254,338.921376,169.364234,91.304934,60.473468,0.569120,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,7503,3180648,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,1,0,...,3114331,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7504,7504,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7505,7505,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7506,7506,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0


In [8]:
print(rg1n_clustered_df)
print(non_buildable)
print(rg1n_clustered_df['RHU_Presence'].unique())
non_buildable

      Unnamed: 0       ID                 barangay_name       city_name  \
0              0  2941561                    Balaytigui         Nasugbu   
1              1  2942942                    Balaytigui         Nasugbu   
2              2  2942943                    Balaytigui         Nasugbu   
3              3  2949831  Barangay 8 (Manuel S. Rojas)  City of Cavite   
4              4  2951212  Barangay 8 (Manuel S. Rojas)  City of Cavite   
...          ...      ...                           ...             ...   
7503        7503  3180648                     Maguibuay      Tagkawayan   
7504        7504  3180649                     Maguibuay      Tagkawayan   
7505        7505  3182031                     Maguibuay      Tagkawayan   
7506        7506  3182032                     Maguibuay      Tagkawayan   
7507        7507  3183413                     Maguibuay      Tagkawayan   

     province_name  flood_probability_value  rain intensity_value  \
0         Batangas            

,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Nearest_RHU,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster
0,0,2941561,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.388247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
1,1,2942942,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.271550,0.001364,0.000792,0.012129,0.006019,0.003191,0.002227,0.435149,0
2,2,2942943,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.391750,4.724720,2.744833,42.027466,20.856238,11.058086,7.717034,0.392740,0
6,6,2951224,Papaya,Nasugbu,Batangas,0.567386,1.355278,0.956776,0,0,...,2971948,22.856154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7,7,2951225,Papaya,Nasugbu,Batangas,0.567386,1.355278,0.956776,0,0,...,2971948,24.245562,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7496,7496,3176550,Alibihaban,San Andres,Quezon,0.912524,2.068573,0.744084,0,0,...,3171020,5.074344,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7504,7504,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7505,7505,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7506,7506,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0


In [9]:
rg1n_clustered_df = rg1n_clustered_df.reset_index(drop=True)
rg1n_clustered_df

,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Nearest_RHU,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster
0,0,2941561,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.388247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
1,1,2942942,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.271550,0.001364,0.000792,0.012129,0.006019,0.003191,0.002227,0.435149,0
2,2,2942943,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,2971948,41.391750,4.724720,2.744833,42.027466,20.856238,11.058086,7.717034,0.392740,0
3,3,2949831,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,2971948,22.516565,17.251052,6.571838,151.930272,75.921898,40.929798,27.108796,0.522489,0
4,4,2951212,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,2971948,21.748555,38.483116,14.660254,338.921376,169.364234,91.304934,60.473468,0.569120,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,7503,3180648,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,1,0,...,3114331,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7504,7504,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7505,7505,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0
7506,7506,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,3114331,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0


### DF Set-up
- Data is normalized

In [10]:
rg1n_clustered_df.columns

Index(['Unnamed: 0', 'ID', 'barangay_name', 'city_name', 'province_name',
       'flood_probability_value', 'rain intensity_value', 'drought_value',
       'buildability_landcov', 'RHU_Presence', 'Road_Presence', 'POI_Presence',
       'Nearest_RHU', 'Distance_to_Nearest_RHU_km', 'popden_chi', 'popden_eld',
       'popden_all', 'popden_wom', 'popden_w_1', 'popden_you', 'HCFAI',
       'Cluster'],
      dtype='object')

In [11]:
'''AGGREGATING THE POPULATION'''
# Assuming df is your DataFrame
vp =  rg1n_clustered_df['popden_chi'] + rg1n_clustered_df['popden_eld'] + rg1n_clustered_df['popden_wom'] + rg1n_clustered_df['popden_you'] + rg1n_clustered_df['popden_w_1'] + rg1n_clustered_df['popden_you']
gen = rg1n_clustered_df['popden_all'] - vp
all = vp + gen
rg1n_clustered_df['total_population'] = all
rg1n_clustered_df

,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster,total_population
0,0,2941561,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,41.388247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
1,1,2942942,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,41.271550,0.001364,0.000792,0.012129,0.006019,0.003191,0.002227,0.435149,0,0.012129
2,2,2942943,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,41.391750,4.724720,2.744833,42.027466,20.856238,11.058086,7.717034,0.392740,0,42.027466
3,3,2949831,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,22.516565,17.251052,6.571838,151.930272,75.921898,40.929798,27.108796,0.522489,0,151.930272
4,4,2951212,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,21.748555,38.483116,14.660254,338.921376,169.364234,91.304934,60.473468,0.569120,1,338.921376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,7503,3180648,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,1,0,...,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7504,7504,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7505,7505,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7506,7506,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000


In [12]:
rg1n_clustered_df.loc[rg1n_clustered_df['total_population'] == 0]
# Drop rows with NaN values
rg1n_clustered_df.dropna(inplace=True)
# x_normalized.dropna(inplace=True)

# Convert column 'x14' from float to int
rg1n_clustered_df['buildability_landcov'] = rg1n_clustered_df['buildability_landcov'].astype(int)
rg1n_clustered_df

,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster,total_population
0,0,2941561,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,41.388247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
1,1,2942942,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,41.271550,0.001364,0.000792,0.012129,0.006019,0.003191,0.002227,0.435149,0,0.012129
2,2,2942943,Balaytigui,Nasugbu,Batangas,0.547602,1.379736,1.061817,0,0,...,41.391750,4.724720,2.744833,42.027466,20.856238,11.058086,7.717034,0.392740,0,42.027466
3,3,2949831,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,22.516565,17.251052,6.571838,151.930272,75.921898,40.929798,27.108796,0.522489,0,151.930272
4,4,2951212,Barangay 8 (Manuel S. Rojas),City of Cavite,Cavite,0.818004,1.093190,0.907145,1,0,...,21.748555,38.483116,14.660254,338.921376,169.364234,91.304934,60.473468,0.569120,1,338.921376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,7503,3180648,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,1,0,...,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7504,7504,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7505,7505,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7506,7506,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000


In [13]:
numeric_rg1_clustered_df = rg1n_clustered_df.drop(['barangay_name', 'city_name', 'province_name', 'Unnamed: 0'], axis=1)
numeric_rg1_clustered_df

,ID,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,Road_Presence,POI_Presence,Nearest_RHU,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster,total_population
0,2941561,0.547602,1.379736,1.061817,0,0,0,0,2971948,41.388247,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
1,2942942,0.547602,1.379736,1.061817,0,0,0,1,2971948,41.271550,0.001364,0.000792,0.012129,0.006019,0.003191,0.002227,0.435149,0,0.012129
2,2942943,0.547602,1.379736,1.061817,0,0,0,0,2971948,41.391750,4.724720,2.744833,42.027466,20.856238,11.058086,7.717034,0.392740,0,42.027466
3,2949831,0.818004,1.093190,0.907145,1,0,0,0,2971948,22.516565,17.251052,6.571838,151.930272,75.921898,40.929798,27.108796,0.522489,0,151.930272
4,2951212,0.818004,1.093190,0.907145,1,0,0,1,2971948,21.748555,38.483116,14.660254,338.921376,169.364234,91.304934,60.473468,0.569120,1,338.921376
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,3180648,0.456619,2.361847,1.006499,1,0,0,0,3114331,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7504,3180649,0.456619,2.361847,1.006499,0,0,0,0,3114331,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7505,3182031,0.456619,2.361847,1.006499,0,0,0,1,3114331,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7506,3182032,0.456619,2.361847,1.006499,0,0,0,0,3114331,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000


In [14]:
# df1[df1['RHU_Presence']==0]
# Count the number of unique values in column 'A'
unique_count = numeric_rg1_clustered_df['Cluster'].nunique()
print("Number of unique values in column 'Cluster':", unique_count)

Number of unique values in column 'Cluster': 2


In [15]:
numeric_rg1_clustered_df.Cluster.unique()
df1 = numeric_rg1_clustered_df[numeric_rg1_clustered_df.isna().any(axis=1)]

df1.loc[df1['RHU_Presence'] == 1, 'buildability_landcov'] = 1
numeric_rg1_clustered_df.loc[numeric_rg1_clustered_df['RHU_Presence'] == 1, 'buildability_landcov'] = 1 # CHANGING AREAS WITH RHUS HAVE A BUILDABILITY SCORE OF 1

In [16]:
# Making id_template for retrieving hex id numbers in the future
id_template = numeric_rg1_clustered_df.astype({'ID' : 'int'})
id_template = id_template.filter(['ID'])
# id_template
y_sample_id = numeric_rg1_clustered_df.filter(['ID', 'Cluster'])
y_sample_id

,ID,Cluster
0,2941561,0
1,2942942,0
2,2942943,0
3,2949831,0
4,2951212,1
...,...,...
7503,3180648,0
7504,3180649,0
7505,3182031,0
7506,3182032,0


In [17]:
y_sample_id[y_sample_id["Cluster"] == 0]

,ID,Cluster
0,2941561,0
1,2942942,0
2,2942943,0
3,2949831,0
6,2951224,0
...,...,...
7503,3180648,0
7504,3180649,0
7505,3182031,0
7506,3182032,0


In [18]:
for i in numeric_rg1_clustered_df.columns:
  print(i)

ID
flood_probability_value
rain intensity_value
drought_value
buildability_landcov
RHU_Presence
Road_Presence
POI_Presence
Nearest_RHU
Distance_to_Nearest_RHU_km
popden_chi
popden_eld
popden_all
popden_wom
popden_w_1
popden_you
HCFAI
Cluster
total_population


In [19]:
x = numeric_rg1_clustered_df.loc[:, ~numeric_rg1_clustered_df.columns.isin(['ID', 'RHU_Presence', 'buildability_landcov',
                                                                            'POI_Presence', 'Road_Presence', 'Nearest_RHU',
                                                                            'Cluster'
                                                                            ])]
x_normalized = (x - x.min()) / (x.max() - x.min())
x_normalized

,flood_probability_value,rain intensity_value,drought_value,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,total_population
0,0.282793,0.504948,0.576803,0.545801,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00
1,0.282793,0.504948,0.576803,0.544262,7.934245e-08,1.185069e-07,8.012604e-08,7.945516e-08,7.618774e-08,7.429536e-08,0.292798,8.012604e-08
2,0.282793,0.504948,0.576803,0.545847,2.749197e-04,4.106236e-04,2.776349e-04,2.753103e-04,2.639887e-04,2.574317e-04,0.158002,2.776349e-04
3,0.423055,0.400079,0.406991,0.296934,1.003796e-03,9.831387e-04,1.003656e-03,1.002198e-03,9.771135e-04,9.043193e-04,0.570404,1.003656e-03
4,0.423055,0.400079,0.406991,0.286806,2.239237e-03,2.193156e-03,2.238926e-03,2.235672e-03,2.179715e-03,2.017328e-03,0.718620,2.238926e-03
...,...,...,...,...,...,...,...,...,...,...,...,...
7503,0.235599,0.864375,0.516070,0.975423,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00
7504,0.235599,0.864375,0.516070,0.973202,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00
7505,0.235599,0.864375,0.516070,0.989401,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00
7506,0.235599,0.864375,0.516070,0.984906,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00


In [20]:
x_normalized['RHU_Presence'] = numeric_rg1_clustered_df['RHU_Presence']
x_normalized['ID'] = numeric_rg1_clustered_df['ID']
x_normalized['buildability_landcov'] = numeric_rg1_clustered_df['buildability_landcov']
x_normalized['Road_Presence'] = numeric_rg1_clustered_df['Road_Presence']
x_normalized['POI_Presence'] = numeric_rg1_clustered_df['POI_Presence']
x_normalized['Nearest_RHU'] = numeric_rg1_clustered_df['Nearest_RHU']
# x_normalized['Neighbors'] = numeric_rg1_clustered_df['Neighbors']
x_normalized['Cluster'] = numeric_rg1_clustered_df['Cluster']
all_normalized = x_normalized
all_normalized.head()

,flood_probability_value,rain intensity_value,drought_value,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,total_population,RHU_Presence,ID,buildability_landcov,Road_Presence,POI_Presence,Nearest_RHU,Cluster
0,0.282793,0.504948,0.576803,0.545801,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,2941561,0,0,0,2971948,0
1,0.282793,0.504948,0.576803,0.544262,7.934245e-08,1.185069e-07,8.012604e-08,7.945516e-08,7.618774e-08,7.429536e-08,0.292798,8.012604e-08,0,2942942,0,0,1,2971948,0
2,0.282793,0.504948,0.576803,0.545847,2.749197e-04,4.106236e-04,2.776349e-04,2.753103e-04,2.639887e-04,2.574317e-04,0.158002,2.776349e-04,0,2942943,0,0,0,2971948,0
3,0.423055,0.400079,0.406991,0.296934,1.003796e-03,9.831387e-04,1.003656e-03,1.002198e-03,9.771135e-04,9.043193e-04,0.570404,1.003656e-03,0,2949831,1,0,0,2971948,0
4,0.423055,0.400079,0.406991,0.286806,2.239237e-03,2.193156e-03,2.238926e-03,2.235672e-03,2.179715e-03,2.017328e-03,0.718620,2.238926e-03,0,2951212,1,0,1,2971948,1


In [21]:
x_normalized = x_normalized.drop(['Cluster'], axis=1)
x_normalized

,flood_probability_value,rain intensity_value,drought_value,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,total_population,RHU_Presence,ID,buildability_landcov,Road_Presence,POI_Presence,Nearest_RHU
0,0.282793,0.504948,0.576803,0.545801,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,2941561,0,0,0,2971948
1,0.282793,0.504948,0.576803,0.544262,7.934245e-08,1.185069e-07,8.012604e-08,7.945516e-08,7.618774e-08,7.429536e-08,0.292798,8.012604e-08,0,2942942,0,0,1,2971948
2,0.282793,0.504948,0.576803,0.545847,2.749197e-04,4.106236e-04,2.776349e-04,2.753103e-04,2.639887e-04,2.574317e-04,0.158002,2.776349e-04,0,2942943,0,0,0,2971948
3,0.423055,0.400079,0.406991,0.296934,1.003796e-03,9.831387e-04,1.003656e-03,1.002198e-03,9.771135e-04,9.043193e-04,0.570404,1.003656e-03,0,2949831,1,0,0,2971948
4,0.423055,0.400079,0.406991,0.286806,2.239237e-03,2.193156e-03,2.238926e-03,2.235672e-03,2.179715e-03,2.017328e-03,0.718620,2.238926e-03,0,2951212,1,0,1,2971948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,0.235599,0.864375,0.516070,0.975423,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3180648,1,0,0,3114331
7504,0.235599,0.864375,0.516070,0.973202,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3180649,0,0,0,3114331
7505,0.235599,0.864375,0.516070,0.989401,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3182031,0,0,1,3114331
7506,0.235599,0.864375,0.516070,0.984906,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3182032,0,0,0,3114331


In [22]:
y_df = numeric_rg1_clustered_df['Cluster'].values
y_df

array([0, 0, 0, ..., 0, 0, 0])

In [23]:
num_features = len(x_normalized.columns)
print("Number of Features: {}".format(num_features))

Number of Features: 18


In [24]:
columns = []
for i in range(num_features):
    columns.append("x{}".format(i))

x_normalized.columns = columns # replacing the name of the columns
x_normalized

,x0,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17
0,0.282793,0.504948,0.576803,0.545801,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,2941561,0,0,0,2971948
1,0.282793,0.504948,0.576803,0.544262,7.934245e-08,1.185069e-07,8.012604e-08,7.945516e-08,7.618774e-08,7.429536e-08,0.292798,8.012604e-08,0,2942942,0,0,1,2971948
2,0.282793,0.504948,0.576803,0.545847,2.749197e-04,4.106236e-04,2.776349e-04,2.753103e-04,2.639887e-04,2.574317e-04,0.158002,2.776349e-04,0,2942943,0,0,0,2971948
3,0.423055,0.400079,0.406991,0.296934,1.003796e-03,9.831387e-04,1.003656e-03,1.002198e-03,9.771135e-04,9.043193e-04,0.570404,1.003656e-03,0,2949831,1,0,0,2971948
4,0.423055,0.400079,0.406991,0.286806,2.239237e-03,2.193156e-03,2.238926e-03,2.235672e-03,2.179715e-03,2.017328e-03,0.718620,2.238926e-03,0,2951212,1,0,1,2971948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,0.235599,0.864375,0.516070,0.975423,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3180648,1,0,0,3114331
7504,0.235599,0.864375,0.516070,0.973202,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3180649,0,0,0,3114331
7505,0.235599,0.864375,0.516070,0.989401,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3182031,0,0,1,3114331
7506,0.235599,0.864375,0.516070,0.984906,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3182032,0,0,0,3114331


In [25]:
y_df = pd.DataFrame(y_df, columns = ['Y_Cluster'])

print(y_df)
print(type(y_df))

      Y_Cluster
0             0
1             0
2             0
3             0
4             1
...         ...
7503          0
7504          0
7505          0
7506          0
7507          0

[7508 rows x 1 columns]
<class 'pandas.core.frame.DataFrame'>


### Upsampling
- use smote to generate data points with '1' (high level) -- for training only!

In [26]:
y_sample = numeric_rg1_clustered_df['Cluster']
y_sample.unique()
rg1n_clustered_df['buildability_landcov'] == 1
# summarize the new class distribution
counter = Counter(y_sample)
print(counter)
# rg1n_clustered_df.loc[1079, ['buildability_landcov']] = [0]
# rg1n_clustered_df.loc[4676, ['buildability_landcov']] = [0]
x_normalized = x_normalized.drop(['x13', 'x17'], axis=1)

Counter({0: 6987, 1: 521})


In [27]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
SS = {1: int(0.7 * len(y_sample))}
# SS = {1: len(y_sample)}
oversample = SMOTE(sampling_strategy=SS, k_neighbors=1)
# oversample = SMOTE(k_neighbors=1)
x_normalized, y_sample = oversample.fit_resample(x_normalized, y_sample)

## Another sampling technique
# Create a pipeline with RandomUnderSampler and SMOTE
# pipeline = Pipeline([
#     ('under_sampling', RandomUnderSampler(sampling_strategy=0.5)),  # Undersample the majority class to 50%
#     ('sampling', SMOTE())  # Apply SMOTE to balance the classes
# ])
# Apply the pipeline to the training data
# X_resampled, y_resampled = pipeline.fit_resample(x_normalized, y_sample)


counter_resampled = Counter(y_sample)
print(f"After resampling: {counter_resampled}")

After resampling: Counter({0: 6987, 1: 5255})


### filtered_low_accessible_df

In [28]:
from pathlib import Path
filepath = Path('/content/drive/MyDrive/MS thesis/[optimize] Training, Testing/CALABARZON/all_normalized.csv')
filepath.parent.mkdir(parents=True, exist_ok=True)
all_normalized.to_csv(filepath, index=False)

In [29]:
all_normalized_df = pd.read_csv(data_dir + '/[optimize] Training, Testing/CALABARZON/all_normalized.csv')
all_normalized_df.head(3)

,flood_probability_value,rain intensity_value,drought_value,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,total_population,RHU_Presence,ID,buildability_landcov,Road_Presence,POI_Presence,Nearest_RHU,Cluster
0,0.282793,0.504948,0.576803,0.545801,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,2941561,0,0,0,2971948,0
1,0.282793,0.504948,0.576803,0.544262,7.934245e-08,1.185069e-07,8.012604e-08,7.945516e-08,7.618774e-08,7.429536e-08,0.292798,8.012604e-08,0,2942942,0,0,1,2971948,0
2,0.282793,0.504948,0.576803,0.545847,2.749197e-04,4.106236e-04,2.776349e-04,2.753103e-04,2.639887e-04,2.574317e-04,0.158002,2.776349e-04,0,2942943,0,0,0,2971948,0


In [30]:
filtered_low_accessible_df = pd.concat([all_normalized], axis=1)
filtered_low_accessible_df = filtered_low_accessible_df[filtered_low_accessible_df['Cluster'] == 0]
filtered_low_accessible_df.head(3)

,flood_probability_value,rain intensity_value,drought_value,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,total_population,RHU_Presence,ID,buildability_landcov,Road_Presence,POI_Presence,Nearest_RHU,Cluster
0,0.282793,0.504948,0.576803,0.545801,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,2941561,0,0,0,2971948,0
1,0.282793,0.504948,0.576803,0.544262,7.934245e-08,1.185069e-07,8.012604e-08,7.945516e-08,7.618774e-08,7.429536e-08,0.292798,8.012604e-08,0,2942942,0,0,1,2971948,0
2,0.282793,0.504948,0.576803,0.545847,2.749197e-04,4.106236e-04,2.776349e-04,2.753103e-04,2.639887e-04,2.574317e-04,0.158002,2.776349e-04,0,2942943,0,0,0,2971948,0


In [31]:
# hex_neighbors = filtered_low_accessible_df[['ID', 'Neighbors']]
# hex_neighbors

In [32]:
filtered_low_accessible_x = filtered_low_accessible_df.drop(['Cluster'], axis=1)
filtered_low_accessible_x = filtered_low_accessible_x.dropna()

In [33]:
# filtered_low_accessible_x = filtered_low_accessible_x.drop(['ID', 'Neighbors', 'Nearest_RHU'], axis=1)
filtered_low_accessible_x = filtered_low_accessible_x.drop(['ID', 'Nearest_RHU'], axis=1)
filtered_low_accessible_df['Cluster'] = filtered_low_accessible_df['Cluster'].astype('int')
filtered_labels_y = filtered_low_accessible_df['Cluster']
filtered_labels_y

0       0
1       0
2       0
3       0
6       0
       ..
7503    0
7504    0
7505    0
7506    0
7507    0
Name: Cluster, Length: 6987, dtype: int64

In [34]:
filtered_low_accessible_df

,flood_probability_value,rain intensity_value,drought_value,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,total_population,RHU_Presence,ID,buildability_landcov,Road_Presence,POI_Presence,Nearest_RHU,Cluster
0,0.282793,0.504948,0.576803,0.545801,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,2941561,0,0,0,2971948,0
1,0.282793,0.504948,0.576803,0.544262,7.934245e-08,1.185069e-07,8.012604e-08,7.945516e-08,7.618774e-08,7.429536e-08,0.292798,8.012604e-08,0,2942942,0,0,1,2971948,0
2,0.282793,0.504948,0.576803,0.545847,2.749197e-04,4.106236e-04,2.776349e-04,2.753103e-04,2.639887e-04,2.574317e-04,0.158002,2.776349e-04,0,2942943,0,0,0,2971948,0
3,0.423055,0.400079,0.406991,0.296934,1.003796e-03,9.831387e-04,1.003656e-03,1.002198e-03,9.771135e-04,9.043193e-04,0.570404,1.003656e-03,0,2949831,1,0,0,2971948,0
6,0.293055,0.495997,0.461480,0.301412,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,2951224,0,0,0,2971948,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,0.235599,0.864375,0.516070,0.975423,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3180648,1,0,0,3114331,0
7504,0.235599,0.864375,0.516070,0.973202,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3180649,0,0,0,3114331,0
7505,0.235599,0.864375,0.516070,0.989401,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3182031,0,0,1,3114331,0
7506,0.235599,0.864375,0.516070,0.984906,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.498924,0.000000e+00,0,3182032,0,0,0,3114331,0


In [35]:
duplicates = filtered_low_accessible_df[filtered_low_accessible_df.duplicated(['ID', 'Cluster'])]
print(duplicates)

Empty DataFrame
Columns: [flood_probability_value, rain intensity_value, drought_value, Distance_to_Nearest_RHU_km, popden_chi, popden_eld, popden_all, popden_wom, popden_w_1, popden_you, HCFAI, total_population, RHU_Presence, ID, buildability_landcov, Road_Presence, POI_Presence, Nearest_RHU, Cluster]
Index: []


In [36]:
y_filtered_labels_id = filtered_low_accessible_df[['ID', 'Cluster']]
y_filtered_labels_id

,ID,Cluster
0,2941561,0
1,2942942,0
2,2942943,0
3,2949831,0
6,2951224,0
...,...,...
7503,3180648,0
7504,3180649,0
7505,3182031,0
7506,3182032,0


In [37]:
y_filtered_labels_id[y_filtered_labels_id['Cluster'] == 1]

,ID,Cluster


## Split and convert Dataset to DataLoader
> For neural network compatibility

In [38]:
## train data
class TrainData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

class TestData(Dataset):
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data
    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]
    def __len__ (self):
        return len(self.X_data)

In [39]:
train_df, test_df = train_test_split(all_normalized_df, test_size=0.3)

## Model Creation
> The model was created with the following parameters:
- number of layers: x
- The model was created to be a classifier to distinguish buildable and non-buildable areas in a region. Buildable areas will become candidate sites

Hyperparameters of developing a BPNN:
1. Loss
2. Number of Layers
3. Optimizer Function
4. Activation Function
5. Batch Size
6. Scheduler

In [40]:
device = "cuda" if torch.cuda.is_available() else "cpu"
class BPNN(nn.Module):
    def __init__(self):
        super(BPNN, self).__init__()
        self.flatten = nn.Flatten()
        self.softmax = nn.Softmax()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(16, 256), # layer 1; input should be the number of inputs sa
            nn.Sigmoid(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 128), # layer 2
            nn.Sigmoid(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 128), # layer 3
            nn.Sigmoid(), # nn.Sigmoid()
            nn.BatchNorm1d(128),
            nn.Linear(128, 64), # layer 4
            nn.Sigmoid(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32), # layer 5
            nn.Sigmoid(),
            nn.BatchNorm1d(32),
            nn.Dropout(),
            nn.Linear(32, 2) # layer 6
        )
    def forward(self, x):
        x = self.flatten(x)
        x = self.linear_relu_stack(x)
        logits = self.softmax(x)
        return logits
model = BPNN().to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3) #1e-4 is also good ish
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5) # 🎯🎯
# scheduler = StepLR(optimizer, step_size=5, gamma=0.1)  # Step size is the number of epochs before decreasing the learning rate
def train(k, epoch, dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      outputs = model(X)
      loss = criterion(outputs, y)
      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      scheduler.step(loss)  # Update learning rate based on validation loss
    if batch % 100 == 0:
        loss, current = loss.item(), batch * len(X)
        train_loss_values.append(loss)
        print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(k, epoch, dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            # Forward pass
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    accuracy = 100*correct
    test_average_loss_values.append(test_loss)
    test_accuracy_values.append(accuracy)
    print(f'Fold: {k+1} | Epoch {epoch+1}: | Loss: {(test_loss):.3f} | Acc: {accuracy:.3f}')
    return test_loss, accuracy

def k_fold_splitter(x,y,BATCH_SIZE):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
  x_train_tensor = torch.tensor(x_train.values, dtype=torch.float)
  x_test_tensor = torch.tensor(x_test.values, dtype=torch.float)
  y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
  y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)
  # Splits X and Y tensors
  train_data = TrainData(x_train_tensor, y_train_tensor)
  test_data = TestData(x_test_tensor, y_test_tensor)
  # Data Loader
  # 13 is good
  # ✈️ GO BACK TO THIS NEXT SEM - something u can optimize pa
  train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
  test_loader = DataLoader(dataset=test_data, batch_size=1)
  return train_loader, test_loader, x_test_tensor, y_test


global outter_all_train_loss
outter_all_train_loss = []

global outter_all_test_average_loss_values
outter_all_test_average_loss_values  = []

global outter_all_test_accuracy
outter_all_test_accuracy = []

global outter_all_pred_per_kfold
outter_all_pred_per_kfold = []

global outter_all_actual_per_kfold
outter_all_actual_per_kfold = []

global outter_actual_in_kfold
outter_actual_in_kfold = []

global outter_pred_in_kfold
outter_pred_in_kfold = []

global hexid_per_kfold
outter_hexid_per_kfold = []

global outter_train_loss_values
outter_train_loss_values = []

# global best_accuracy
folds = range(10)
best_accuracy = 0.0
best_model = None
# Add a list of batch sizes to experiment with
batch_sizes = [16, 32, 64, 128]
best_accuracy_overall = 0.0
best_model_overall = None
best_k_overall = None
best_batch_size = None
best_batch_size_avg_collection = None

best_all_train_loss = None
best_all_test_average_loss_values = None
best_all_test_accuracy = None
best_all_pred_per_kfold = None
best_all_actual_per_kfold = None
best_actual_in_kfold = None
best_pred_in_kfold = None
best_hexid_per_kfold = None
best_train_loss_values = None

BPNN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (softmax): Softmax(dim=None)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=16, out_features=256, bias=True)
    (1): Sigmoid()
    (2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): Sigmoid()
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): Sigmoid()
    (8): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Linear(in_features=128, out_features=64, bias=True)
    (10): Sigmoid()
    (11): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): Linear(in_features=64, out_features=32, bias=True)
    (13): Sigmoid()
    (14): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): Dropout(p=0.5, inp

# Driver Code
- Train and Test the model
- inputs: normalized features of Region 1 (with upsampled labels)
- outputs: Model with the best performance during a specific k-fold

- https://towardsdatascience.com/pytorch-tabular-binary-classification-a0368da5bb89

In [ ]:
for k in folds:
  train_loader, test_loader, x_test_tensor, y_test = k_fold_splitter(x_normalized, y_sample)
  test_loss_values = []
  test_average_loss_values = []
  test_accuracy_values = []

  counter = 0  # Counter for consecutive epochs with no improvement
  prev_val_loss = float('inf')  # Initialize previous validation loss
  prev_accuracy = 0.0            # Initialize previous accuracy
  patience = 10  # Number of epochs with no improvement to wait before stopping

  model = BPNN().to(device)
  epochs = 250 ## change to best epoch pls
  print(f"=================== K = {k+1} ===================\n")

  for epoch in range(epochs):
      train(k,epoch,train_loader, model, criterion, optimizer)
      val_loss, val_accuracy = test(k,epoch,test_loader, model, criterion)
      if val_accuracy > best_accuracy:
          best_accuracy = val_accuracy
          print("BEST: ", best_accuracy)
          best_model = model.state_dict().copy()  # Save a copy of the model's state
          # print(f"Best model is {best_model} from k = {k}")
          counter = 0 # Reset counter if there's improvement
      else:
          counter += 1
      if counter >= patience:
          print(f'No improvement for {patience} consecutive epochs. Stopping early.')
          break

  predictions = model(x_test_tensor)
  predictions = F.softmax(predictions)
  predictions = torch.argmax(predictions, dim=1)
  pred_in_kfold.append(predictions)
  actual_y = y_test # actual y values per k-fold
  # loss
  all_train_loss.append(train_loss_values)
  all_test_average_loss_values.append(test_average_loss_values)
  all_test_accuracy.append(test_accuracy_values) # accuracy
  # y pred and actual y
  actual_in_kfold.append(actual_y)
  all_pred_per_kfold.append(pred_in_kfold)
  all_actual_per_kfold.append(actual_y.tolist())

  print(f"BEST accuracy in {k}-fold: ")
  print(f"Saving... best_model with accuracy of {best_accuracy}")
  torch.save(best_model, '/content/drive/MyDrive/MS thesis/[optimize] Training, Testing/best_model2.pt')

print("Done!")

## Driver Results

In [ ]:
i = 0
for hexlist in hexid_per_kfold:
  new_hexlist = []
  for j in hexlist:
    temp = int(j)
    new_hexlist.append(temp)
  hexid_per_kfold[i] = new_hexlist
  i +=1

results = pd.DataFrame()
results['pred_y'] = all_pred_per_kfold[0][0]
results['actual_y'] = all_actual_per_kfold[0][0]
results

In [ ]:
# model = NeuralNetwork()
# model.load_state_dict(torch.load("model.pth"))

from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')
confusion_per_k = []

for k in folds:
  mlcm = multilabel_confusion_matrix(actual_in_kfold[k], pred_in_kfold[k])
  print(f'=============== K = {k} ================')
  for cm in mlcm:
    tp = cm[0][0]
    tn = cm[1][1]
    fp = cm[0][1]
    fn = cm[1][0]

    mcc = ((tn * tp) - (fn * fp)) / math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))
    f1 = tp / (tp + (0.5 * (fp + fn)))

  print(classification_report(actual_in_kfold[k], pred_in_kfold[k]))
  print('MCC: {}'.format(mcc))
  print('F1: {}'.format(f1))
    # if 0 accuracy is so bad, F1 is how much the predictions are right
    # might need to fix the amount under each label

In [ ]:
def Average(lst):
  return sum(lst) / len(lst)

K = folds
avg_accuracy = []
for i in all_test_accuracy:
  avg = Average(i)
  avg_accuracy.append(avg)
print(avg_accuracy)

In [ ]:
plt.bar(K, avg_accuracy, color ='darkblue', width = 0.6)
plt.xlabel("K-fold")
plt.ylabel("Average Accuracy")
plt.title("Average accuracy per K-fold")
plt.show()

In [ ]:
# cf_matrix = confusion_matrix(actual_in_kfold[0], pred_in_kfold[0])
# plt.figure(figsize = (12,7))
# sns.heatmap(cf_matrix, annot=True)
# # Loop over each fold
for k in folds:
    cf_matrix = confusion_matrix(actual_in_kfold[k], pred_in_kfold[k])
    # Plot confusion matrix
    plt.figure(figsize=(12, 7))
    sns.heatmap(cf_matrix, annot=True, fmt='d', cmap='Blues',
               xticklabels=['Predicted Negative', 'Predicted Positive'],
               yticklabels=['Actual Negative', 'Actual Positive'])
    plt.title(f'Confusion Matrix - Fold {k+1}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

# Candidate sites
> This section aims to look for potential optimal locations to put up RHUs. We will call these locations candidate sites. We will use the BP Neural Network that was trained in the section above to find these locations.

#### The results show that there are **964 candidate sites**. Out of the 6508  locations, we were able to narrow this down to 14% of the Region's LOW ACCESSIBLE AREAS (6508) can have RHUs placed on them.

In [50]:
# Step 1: Load the saved model
candidate_nn = BPNN().to(device)
modelpath = "/content/drive/MyDrive/MS thesis/[optimize] Training, Testing/modified_folds_model.pt" # ⭐👑
# modelpath = "/content/drive/MyDrive/MS thesis/[optimize] Training, Testing/bpnn_model.pt" # ⭐👑

# model = BPNN()  # Instantiate your model class
candidatenn = BPNN().to(device)
candidatenn.load_state_dict(torch.load(modelpath))
candidatenn.eval()  # Set the model to evaluation mode

def low_k_fold_splitter(x,y,low_x,low_y):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
  # actual labels for testing
  y_test = low_y
  x_train_tensor = torch.tensor(x_train.values, dtype=torch.float)
  x_test_tensor = torch.tensor(low_x.values, dtype=torch.float)
  y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
  y_test_tensor = torch.tensor(low_y.values, dtype=torch.long)

  # Splits X and Y tensors
  train_data = TrainData(x_train_tensor, y_train_tensor)
  test_data = TestData(x_test_tensor, y_test_tensor)

  # Data Loader
  BATCH_SIZE = 16 # ✈️ GO BACK TO THIS NEXT SEM - something u can optimize pa
  train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)
  test_loader = DataLoader(dataset=test_data, batch_size=1)
  return train_loader, test_loader, x_test_tensor, y_test

# global all_train_loss
all_train_loss = []
# global all_test_average_loss_values
all_test_average_loss_values  = []
# global all_test_accuracy
all_test_accuracy = []
# global all_pred_per_kfold
all_pred_per_kfold = []
# global all_actual_per_kfold
all_actual_per_kfold = []
# global actual_in_kfold
actual_in_kfold = []
# global pred_in_kfold
pred_in_kfold = []
# global hexid_per_kfold
hexid_per_kfold = []
best_accuracy = 0.0
modelpatience =20  # Number of epochs with no improvement to wait before stopping

# for k in folds:
#   train_loader, test_loader, x_test_tensor, y_test = low_k_fold_splitter(x_normalized, y_sample, filtered_low_accessible_x, filtered_labels_y)
#   train_loss_values = []
#   test_loss_values = []
#   test_average_loss_values = []
#   test_accuracy_values = []
#   counter = 0  # Counter for consecutive epochs with no improvement
#   prev_val_loss = float('inf')  # Initialize previous validation loss
#   prev_accuracy = 0.0            # Initialize previous accuracy
#   epochs = 100 ## change to best epoch pls
#   print(f"=================== K = {k+1} ===================\n")

#   for epoch in range(epochs):
#       train(k,epoch,train_loader, candidatenn, criterion, optimizer)
#       val_loss, val_accuracy = test(k,epoch,test_loader, candidatenn, criterion)
#       if val_accuracy > best_accuracy:
#           best_accuracy = val_accuracy
#           counter = 0 # Reset counter if there's improvement
#       else:
#           counter += 1
#       if counter >= modelpatience:
#           print(f'No improvement for {modelpatience} consecutive epochs. Stopping early.')
#           break

#   predictions = candidatenn(x_test_tensor)
#   predictions = F.softmax(predictions)
#   predictions = torch.argmax(predictions, dim=1)
#   pred_in_kfold.append(predictions)
#   actual_y = y_test # actual y values per k-fold
#   all_train_loss.append(train_loss_values)# loss
#   all_test_average_loss_values.append(test_average_loss_values)# loss
#   all_test_accuracy.append(test_accuracy_values) # accuracy
#   actual_in_kfold.append(actual_y) # y pred and actual y
#   all_pred_per_kfold.append(pred_in_kfold) # y pred and actual y
#   all_actual_per_kfold.append(actual_y.tolist()) # y pred and actual y

#   print("Done!")

# Load the trained model and perform predictions
for k in folds:
  train_loader, test_loader, x_test_tensor, y_test = low_k_fold_splitter(x_normalized, y_sample, filtered_low_accessible_x, filtered_labels_y)
  test_loss_values = []
  test_average_loss_values = []
  test_accuracy_values = []
  counter = 0

  # Load the model
  candidatenn.load_state_dict(torch.load(modelpath))
  epochs = 100
  for t in range(epochs):
    print(f"-------------------------------\nEpoch {t+1}\n-------------------------------")
    val_loss, val_accuracy = test(k,t,test_loader, candidatenn, criterion)
    if val_accuracy > best_accuracy:
      best_accuracy = val_accuracy
      counter = 0 # Reset counter if there's improvement
    else:
      counter += 1
    if counter >= modelpatience:
      print(f'No improvement for {counter} consecutive epochs. Stopping early.')
      break
  candidatenn.eval()  # Set the model to evaluation mode

  # Perform predictions
  predictions = candidatenn(x_test_tensor)
  predictions = F.softmax(predictions, dim=1)
  predictions = torch.argmax(predictions, dim=1)
  pred_in_kfold.append(predictions)
  actual_y = y_test  # actual y values per k-fold

  all_test_average_loss_values.append(test_average_loss_values)  # loss
  all_test_accuracy.append(test_accuracy_values)  # accuracy
  actual_in_kfold.append(actual_y)  # y pred and actual y
  all_pred_per_kfold.append(pred_in_kfold)  # y pred and actual y
  all_actual_per_kfold.append(actual_y.tolist())  # y pred and actual y

  print("Done with predictions for fold", k+1)

-------------------------------
Epoch 1
-------------------------------


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Fold: 1 | Epoch 1: | Loss: 0.842 | Acc: 19.665
-------------------------------
Epoch 2
-------------------------------
Fold: 1 | Epoch 2: | Loss: 0.842 | Acc: 19.665
-------------------------------
Epoch 3
-------------------------------
Fold: 1 | Epoch 3: | Loss: 0.842 | Acc: 19.665
-------------------------------
Epoch 4
-------------------------------
Fold: 1 | Epoch 4: | Loss: 0.842 | Acc: 19.665
-------------------------------
Epoch 5
-------------------------------
Fold: 1 | Epoch 5: | Loss: 0.842 | Acc: 19.665
-------------------------------
Epoch 6
-------------------------------
Fold: 1 | Epoch 6: | Loss: 0.842 | Acc: 19.665
-------------------------------
Epoch 7
-------------------------------
Fold: 1 | Epoch 7: | Loss: 0.842 | Acc: 19.665
-------------------------------
Epoch 8
-------------------------------
Fold: 1 | Epoch 8: | Loss: 0.842 | Acc: 19.665
-------------------------------
Epoch 9
-------------------------------
Fold: 1 | Epoch 9: | Loss: 0.842 | Acc: 19.665
-

In [51]:
# results['hex_id'] = hexid_per_kfold[0]
results = pd.DataFrame()
results['pred_y'] = all_pred_per_kfold[0][0]
results['actual_y'] = all_actual_per_kfold[0][0]

# candidates = pd.concat([results, y_filtered_labels_id], axis=1)
# candidates = candidates.drop(['index'], axis=1)
results[results['pred_y'] == 1]

# Reset index of both DataFrames
y_filtered_labels_id.reset_index(drop=True, inplace=True)
results.reset_index(drop=True, inplace=True)

# Merge the DataFrames side by side
merged_df = pd.concat([y_filtered_labels_id, results], axis=1)

# Display the merged DataFrame
print(merged_df)

candidates = merged_df[merged_df['pred_y'] == 1]
candidate_sites = candidates
len(candidate_sites)

all = len(results)
print(all)
buildable = len(candidate_sites)
print(buildable)
build_rhu_landscape = buildable/all
print(build_rhu_landscape) # this tells us how much of the area is a candidate site

# Export the dataset
save_dir = "/content/drive/MyDrive/MS thesis/[optimize] Training, Testing/CALABARZON"
# hex_neighbors.to_csv(save_dir + '/Neighbors.csv', index=False)
candidate_sites.to_csv(save_dir + '/candidate_sites.csv', index=False)
filtered_low_accessible_df.to_csv(save_dir + '/PCF_data.csv', index=False)

           ID  Cluster  pred_y  actual_y
0     2941561        0       0         0
1     2942942        0       0         0
2     2942943        0       0         0
3     2949831        0       1         0
4     2951224        0       0         0
...       ...      ...     ...       ...
6982  3180648        0       1         0
6983  3180649        0       0         0
6984  3182031        0       0         0
6985  3182032        0       0         0
6986  3183413        0       0         0

[6987 rows x 4 columns]
6987
5613
0.8033490768570202


# PCF calculations
**Goal:** Get the buildable areas with their populations

**Remember, we want to target areas with HIGH POPULATION and LIMITED to NO RHU ACCESS!**

#### ***Recommended number of facilities is 5 for Region 4a.***
dont forget you will be using this value in optimize-2 when you get the optimal number of facilities


In [ ]:
# rg1n_clustered_df.drop(rg1n_clustered_df.index[rg1n_clustered_df['Cluster'] == 1], inplace = True)
# rg1n_clustered_df.drop(rg1n_clustered_df.index[rg1n_clustered_df['popden_all'] == 0], inplace = True)

hex_with_rhu = rg1n_clustered_df[rg1n_clustered_df['RHU_Presence'] == 1]
hex_with_rhu
buildable = rg1n_clustered_df[rg1n_clustered_df['buildability_landcov'] == 1]
buildable
non_buildable.shape # (5292, 23)
buildable.shape # (2216, 24)

(2215, 23)

In [ ]:
high_clustered = buildable[buildable['Cluster'] == 1]
high_clustered[high_clustered['buildability_landcov'] == 1] # look for buildable areas in high clustered areas
high_clustered[high_clustered['buildability_landcov'] == 0] # look for non buildable areas in high clustered areas
high_clustered['buildability_landcov'].unique() # looking for unique values. If its one, all places in this df are buiildable for RHUs

# df with RHUs + df with buildable areas == SABLAYAN
# using this to compute for the total population
frames = [hex_with_rhu, rg1n_clustered_df]
rg1_PCF_df = pd.concat(frames)
rg1_PCF_df

,Unnamed: 0,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,...,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster,total_population
382,382,2970565,Gomez-Zamora (Pob.),Naic,Cavite,0.526595,1.477501,1.105369,0,1,...,0.000000,658.745788,812.843865,8049.010116,4094.087924,2220.324693,1499.574238,0.501549,0,8049.010116
659,659,2981629,Calumpang Cerca,Indang,Cavite,0.249117,1.712915,1.203267,0,1,...,0.000000,469.509009,517.216406,5661.398245,2885.228780,1531.448103,1089.525536,0.423709,0,5661.398245
783,783,2985759,Tabon III,Kawit,Cavite,0.771205,1.124705,1.046612,0,1,...,0.000000,2962.826285,1935.937180,27920.958940,14055.187699,7679.249704,5337.458080,0.504555,0,27920.958940
804,804,2985780,Miguel Mojica,Mendez (Mendez-Nunez),Cavite,0.163139,1.702739,1.129446,0,1,...,0.000000,1188.454618,1366.199792,12861.468384,6549.532980,3349.889788,2231.523366,0.441989,0,12861.468384
892,892,2988523,Toclong I-C,City of Imus,Cavite,0.807639,1.181400,0.962089,0,1,...,0.000000,5208.886668,3445.871289,54059.595910,27916.684100,15925.633852,10166.067474,0.481689,0,54059.595910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7503,7503,3180648,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,1,0,...,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7504,7504,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7505,7505,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7506,7506,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,...,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000


In [ ]:
rg1_PCF_df.reset_index(drop=True, inplace=True)
rg1_PCF_df.drop(columns=['Unnamed: 0'], inplace=True)
total_rgn1_population = 0
def PCF(total_population):
  PCF = total_population / 20000
  return int(PCF)

In [ ]:
a = 1.65 # or 13042??
pd = rg1_PCF_df['popden_all']
rg1_PCF_df['total_population'] = pd * a
rg1_PCF_df

,ID,barangay_name,city_name,province_name,flood_probability_value,rain intensity_value,drought_value,buildability_landcov,RHU_Presence,Road_Presence,...,Distance_to_Nearest_RHU_km,popden_chi,popden_eld,popden_all,popden_wom,popden_w_1,popden_you,HCFAI,Cluster,total_population
0,2970565,Gomez-Zamora (Pob.),Naic,Cavite,0.526595,1.477501,1.105369,0,1,1,...,0.000000,658.745788,812.843865,8049.010116,4094.087924,2220.324693,1499.574238,0.501549,0,13280.866691
1,2981629,Calumpang Cerca,Indang,Cavite,0.249117,1.712915,1.203267,0,1,0,...,0.000000,469.509009,517.216406,5661.398245,2885.228780,1531.448103,1089.525536,0.423709,0,9341.307104
2,2985759,Tabon III,Kawit,Cavite,0.771205,1.124705,1.046612,0,1,1,...,0.000000,2962.826285,1935.937180,27920.958940,14055.187699,7679.249704,5337.458080,0.504555,0,46069.582251
3,2985780,Miguel Mojica,Mendez (Mendez-Nunez),Cavite,0.163139,1.702739,1.129446,0,1,0,...,0.000000,1188.454618,1366.199792,12861.468384,6549.532980,3349.889788,2231.523366,0.441989,0,21221.422834
4,2988523,Toclong I-C,City of Imus,Cavite,0.807639,1.181400,0.962089,0,1,0,...,0.000000,5208.886668,3445.871289,54059.595910,27916.684100,15925.633852,10166.067474,0.481689,0,89198.333252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7534,3180648,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,1,0,0,...,73.966609,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7535,3180649,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,0,...,73.798172,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7536,3182031,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,0,...,75.026516,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000
7537,3182032,Maguibuay,Tagkawayan,Quezon,0.456619,2.361847,1.006499,0,0,0,...,74.685667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0,0.000000


In [ ]:
rg1_PCF_df.columns
total_rgn1_population = rg1_PCF_df['total_population'].sum()
# total_rgn1_population = rg1_PCF_df['popden_all'].sum()
print(round(total_rgn1_population, 2))
print(PCF(total_rgn1_population))

27369199.89
1368
